## Case Study по модулю SQL
### Назирзода Рахимджон 04-03-2025
### Блок 1. Создание схемы и таблиц
**1.В базе данных создайте новую схему adv_works.**

**2.На основе данных из файла создайте в схеме adv_works таблицы и соедините их между собою по основным и внешним ключам.**


In [ ]:
import json
import pandas as pd
from sqlalchemy import text

from connect import set_connection

In [37]:
SCHEMA = "adv_works"

In [35]:
def create_tables():
    try:
        
        with open('queries/cas_study_on_sql.sql', encoding='utf-8') as f:
            tables_query = f.read()
        
        # создаём схему и таблицы
        with set_connection() as psg:
            psg.execute(text(tables_query))
            psg.commit()
        
        print("Tables created successfully")
    except Exception as e:
        print(e)

In [36]:
create_tables()

Tables created successfully


In [38]:
def read_xl(sheet_name, columns_dict):
    temp_df = pd.read_excel(
        'adventure_works.xlsx',
        sheet_name=sheet_name,
        usecols=columns_dict.keys()
    ).rename(columns=columns_dict)
    return temp_df

In [39]:
with open('columns.json') as f:
    tables_dict = json.load(f) 

In [40]:
def insert_to_db(temp_df, tbl_name):
    with set_connection() as psg:
        temp_df.to_sql(
            schema=SCHEMA,
            name=tbl_name,
            con=psg,
            index=False,
            if_exists='append'
        )

In [41]:
def etl(sheet_name, columns_dict, tbl_name):
    print(f"inserting data to {tbl_name}...")
    temp_df = read_xl(sheet_name, columns_dict)
    insert_to_db(temp_df, tbl_name)

In [42]:
for k, v in tables_dict.items():
    etl(k, v["columns"], v["table_name"])

inserting data to customers...
inserting data to territory...
inserting data to product_category...
inserting data to product_subcategory...
inserting data to products...
inserting data to sales...


### Блок 2. Аналитические задачи
#### Секция 1. Анализ клиентов

**1. Сегментация по доходу:** Посчитайте средний годовой личный доход клиентов (YearlyIncome) в разбивке по роду деятельности (Occupation). Итоговая таблица должна содержать следующие поля: occupation, number_of_customers, avg_income.



In [27]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

In [28]:
connection_url = URL.create(
    drivername="postgresql+psycopg2",
    username="postgres",
    password="123",  
    host="localhost",
    port=5432,
)

engine = create_engine(connection_url)
conn = engine.connect()

print("Успешное подключение к PostgreSQL!")

Успешное подключение к PostgreSQL!


In [9]:
query = """
select 
    occupation,
    count(*) as number_of_customers,
    round(avg(yearly_income), 2) as avg_income
from adv_works.customers
group by 
	occupation
order by 
	avg_income desc;
"""

income_df = pd.read_sql(query, conn)

income_df

,occupation,number_of_customers,avg_income
0,Management,3075,92325.20
1,Professional,5520,74184.78
2,Skilled Manual,4577,51715.10
3,Clerical,2928,30710.38
4,Manual,2384,16451.34


**Короткий вывод:** Данный код выполнил сегментацию клиентов по роду деятельности, считая количество клиентов для каждой категории и средный годовой доход этих клиентов.

**2. Семейный профиль:** Посчитайте долю (в процентах) клиентов с детьми и долю клиентов без детей. Итоговая таблица должна содержать следующие поля: has_children (где 1 означает - имеет детей и 0 - не имеет детей), pct_of_customer_base.

In [10]:
query = """
select 
    has_children,
    round(100.0 * count(*) / total.count, 2) as pct_of_customer_base
from (
    select 
        case 
            when number_of_children > 0 then 1
            else 0
        end as has_children
    from adv_works.customers
) as subquery,
(
    select count(*) as count from adv_works.customers
) as total
group by 
	has_children, total.count;
"""

family_df = pd.read_sql(query, conn)

family_df

,has_children,pct_of_customer_base
0,1,39.86
1,0,60.14


**Короткий вывод:** Код определил процент клиентов с детьми и без детей. 

**3. Высокодоходные клиенты:** Сформируйте список топ 10 клиентов с наибольшей суммой покупок (поле SalesAmount). Итоговая таблица должна содержать следующие поля: customer_key, customer_name, total_purchase.

In [11]:
query = """
select 
    c.customer_key,
    c.full_name,
    round(sum(s.sales_amount), 2) as total_purchase
from adv_works.customers c
join adv_works.sales s 
	on c.customer_key = s.customer_key
group by 
	c.customer_key, c.full_name
order by 
	total_purchase desc
limit 10;
"""

high_inc_cus_df = pd.read_sql(query, conn)

high_inc_cus_df

,customer_key,full_name,total_purchase
0,12301,Nichole Nara,13295.38
1,12132,Kaitlyn Henderson,13294.27
2,12308,Margaret He,13269.27
3,12131,Randall Dominguez,13265.99
4,12300,Adriana Gonzalez,13242.70
5,12321,Rosa Hu,13215.65
6,12124,Brandi Gill,13195.64
7,12307,Brad She,13173.19
8,12296,Francisco Sara,13164.64
9,11433,Maurice Shan,12909.67


**Короткий вывод:** Запрос формировал список топ 10 клиентов с наибольшей суммой покупкой включая полное имя и фамилия и сумма покупки.

**4. Влияние семейного положения:** Посчитайте среднюю сумму продаж в разбивке по семейному положению клиентов (MaritalStatus) и определите насколько сильно различаются средние суммы между двумя группами. Итоговая таблица должна содержать следующие поля: year, marital_status, avg_sales_amount.

In [14]:
query = """
select 
    extract(year from s.order_date) as year,
    c.marital_status,
    round(avg(s.sales_amount), 2) as avg_sales_amount
from adv_works.customers c
join adv_works.sales s 
	on c.customer_key = s.customer_key
group by year, c.marital_status
order by year, avg_sales_amount desc;
"""

marital_df = pd.read_sql(query, conn)

marital_df

,year,marital_status,avg_sales_amount
0,2001.0,M,3245.03
1,2001.0,S,3203.84
2,2002.0,S,2482.13
3,2002.0,M,2397.07
4,2003.0,S,427.78
5,2003.0,M,378.56
6,2004.0,S,318.05
7,2004.0,M,290.64


**Короткий вывод:** Анализировали семейного положения клиентов на среднюю сумму продаж, что позволяет увидеть различия в расходах между клиентами с разным семейным положением в каждый год.

#### Секция 2. Анализ продаж


**1. Ежемесячные продажи:** Создайте отчёт продаж по месяцам за последние 2 года (2003, 2004). Итоговая таблица должна содержать следующие поля: year, monthkey, month_name, sales_count (количество продаж), sales_amount.

In [15]:
query = """
select 
    extract(year from s.order_date) as year,
    to_char(s.order_date, 'YYYYMM') as monthkey,
    to_char(s.order_date, 'Month') as month_name,
    count(*) as sales_count,
    round(sum(s.sales_amount), 2) as sales_amount
from adv_works.sales s
where extract(year from s.order_date) in (2003, 2004)
group by year, monthkey, month_name
order by year, monthkey;
"""

monthly_sales_df = pd.read_sql(query, conn)

monthly_sales_df

,year,monthkey,month_name,sales_count,sales_amount
0,2003.0,200301,January,244,438865.17
1,2003.0,200302,February,272,489090.34
2,2003.0,200303,March,272,485574.79
3,2003.0,200304,April,294,506399.27
4,2003.0,200305,May,335,562772.56
5,2003.0,200306,June,321,554799.23
6,2003.0,200307,July,1411,886668.84
7,2003.0,200308,August,3819,847413.51
8,2003.0,200309,September,3885,1010258.13
9,2003.0,200310,October,4146,1080449.58


**Короткий вывод:** Формировали отчет по ежемесячным продажам за 2003 и 2004 годы, что позволяет увидеть продажи по месяцам в течение последних двух лет.

**2.Продажи по регионам:** Посчитайте сумму продаж в разбивке по регионам. Итоговая таблица должна содержать следующие поля: region, sales_count, sales_amount.

In [16]:
query = """
select 
    t.region,
    count(*) as sales_count,
    round(sum(s.sales_amount), 2) as sales_amount
from adv_works.sales s
join adv_works.territory t on s.territory_key = t.territory_key
group by t.region
order by sales_amount desc;
"""

sales_per_region_df = pd.read_sql(query, conn)

sales_per_region_df

,region,sales_count,sales_amount
0,Australia,13345,9061000.58
1,Southwest,12265,5718150.81
2,Northwest,8993,3649866.55
3,United Kingdom,6906,3391712.21
4,Germany,5625,2894312.34
5,France,5558,2644017.71
6,Canada,7620,1977844.86
7,Southeast,39,12238.85
8,Northeast,27,6532.47
9,Central,20,3000.83


**Короткий вывод:** Анализировали сумму продаж по регионам, что позволяет выделить регионы с наибольшими продажами.

#### Секция 3. Анализ продуктов

**1. Доля продаж:** Посчитайте какую долю от общих продаж составляет каждая категория продуктов. Итоговая таблица должна содержать следующие поля: year, product_key, product_category_key, english_product_category_name, sales_amount, pct_of_total_sales.

In [17]:
query = """
select 
    extract(year from s.order_date) as year,
    p.product_key,
    pc.category_key as product_category_key,
    pc.category_name,
    round(sum(s.sales_amount), 2) as sales_amount,
    round(100.0 * sum(s.sales_amount) / total.total_sales, 2) as pct_of_total_sales
from adv_works.sales s
join adv_works.products p 
    on s.product_key = p.product_key
join adv_works.product_subcategory psc 
    on p.subcategory_key = psc.subcategory_key
join adv_works.product_category pc 
    on psc.category_key = pc.category_key
cross join (
    select sum(sales_amount) as total_sales from adv_works.sales
) as total
group by year, p.product_key, 
    pc.category_key, 
    pc.category_name, 
    total.total_sales
order by year, pct_of_total_sales desc;
"""

sales_df = pd.read_sql(query, conn)

sales_df

,year,product_key,product_category_key,category_name,sales_amount,pct_of_total_sales
0,2001.0,310,1,Bikes,593992.82,2.02
1,2001.0,312,1,Bikes,547475.31,1.86
2,2001.0,311,1,Bikes,500957.80,1.71
3,2001.0,314,1,Bikes,486644.72,1.66
4,2001.0,313,1,Bikes,472331.64,1.61
...,...,...,...,...,...,...
311,2004.0,463,3,Clothing,6685.77,0.02
312,2004.0,484,4,Accessories,4173.75,0.01
313,2004.0,481,3,Clothing,1474.36,0.01
314,2004.0,480,4,Accessories,4202.15,0.01


**Короткий вывод:** Рассчитывали долю продаж каждой категории продуктов от общих продаж, что позволяет увидеть, как каждый продукт и категория вносят вклад в общие продажи.

**2. Самые продаваемые продукты:** Определите топ 5 продуктов с наибольшей суммой продаж. Итоговая таблица должна содержать следующие поля: product_key, product_name, english_product_category_name, sales_amount

In [18]:
query = """
select 
    p.product_key,
    p.product_name,
    pc.category_name,
    round(sum(s.sales_amount), 2) as sales_amount
from adv_works.sales s
join adv_works.products p on s.product_key = p.product_key
join adv_works.product_subcategory psc on p.subcategory_key = psc.subcategory_key
join adv_works.product_category pc on psc.category_key = pc.category_key
group by p.product_key, p.product_name, pc.category_name
order by sales_amount desc
limit 5;
"""

top_products_df = pd.read_sql(query, conn)

top_products_df

,product_key,product_name,category_name,sales_amount
0,312,"Road-150 Red, 48",Bikes,1205876.99
1,310,"Road-150 Red, 62",Bikes,1202298.72
2,313,"Road-150 Red, 52",Bikes,1080637.54
3,314,"Road-150 Red, 56",Bikes,1055589.65
4,311,"Road-150 Red, 44",Bikes,1005493.87


**Короткий вывод:** Определили топ 5 продуктов с наибольшей суммой продаж по убыванию суммы продаж, и ограничены пятью наиболее продаваемыми продуктами.

**3. Маржа от продаж:** Посчитайте разницу между суммой продаж (SalesAmount) за минусом себестоимости (TotalProductCost), налогов (*TaxAmt) и расходов на доставку (Freight) по каждому продукту в разбивке по годам и месяцам. Итоговая таблица должна содержать следующие поля: year, monthkey, month_name, product_key, product_name, sales_amount, total_product_cost, tax_amt, freight, margin, margin_pct (маржа как процент от суммы продаж).

In [19]:
query = """
select 
    extract(year from s.order_date) as year,
    to_char(s.order_date, 'YYYYMM') as monthkey,
    to_char(s.order_date, 'Month') as month_name,
    p.product_key,
    p.product_name,
    round(sum(s.sales_amount), 2) as sales_amount,
    round(sum(s.total_product_cost), 2) as total_product_cost,
    round(sum(s.tax_amount), 2) as tax_amt,
    round(sum(s.freight), 2) as freight,
    round(sum(
    	s.sales_amount - s.total_product_cost - s.tax_amount - s.freight
    	), 2) as margin,
    round(100.0 * sum(
    				s.sales_amount - s.total_product_cost - s.tax_amount - s.freight
    				) / sum(
    				s.sales_amount
    				), 2) as margin_pct
from adv_works.sales s
join adv_works.products p on s.product_key = p.product_key
group by year, monthkey, month_name, p.product_key, p.product_name
order by year, monthkey, margin desc;
"""

sales_margins_df = pd.read_sql(query, conn)

sales_margins_df

,year,monthkey,month_name,product_key,product_name,sales_amount,total_product_cost,tax_amt,freight,margin,margin_pct
0,2001.0,200107,July,312,"Road-150 Red, 48",100191.56,60796.24,8015.32,2504.79,28875.21,28.82
1,2001.0,200107,July,311,"Road-150 Red, 44",82300.21,49939.77,6584.02,2057.51,23718.92,28.82
2,2001.0,200107,July,310,"Road-150 Red, 62",78721.94,47768.47,6297.76,1968.05,22687.66,28.82
3,2001.0,200107,July,314,"Road-150 Red, 56",53674.05,32569.41,4293.92,1341.85,15468.86,28.82
4,2001.0,200107,July,313,"Road-150 Red, 52",42939.24,26055.53,3435.14,1073.48,12375.09,28.82
...,...,...,...,...,...,...,...,...,...,...,...
1895,2004.0,200407,July,481,"Racing Socks, M",152.83,57.16,12.23,3.82,79.62,52.10
1896,2004.0,200407,July,488,"Short-Sleeve Classic Jersey, S",593.89,457.30,47.51,14.85,74.24,12.50
1897,2004.0,200407,July,490,"Short-Sleeve Classic Jersey, L",431.92,332.58,34.55,10.80,53.99,12.50
1898,2004.0,200407,July,479,Road Bottle Cage,89.90,33.62,7.19,2.25,46.84,52.10


**Короткий вывод:** Рассчитывали маржу от продаж для каждого продукта по годам и месяцам, что позволяет увидеть, какие продукты и месяцы имеют наибольшую маржу.

#### Секция 4. Анализ трендов

**1. Квартальный рост:** Посчитайте сумму продаж за каждый квартал и их процентное изменение по топ 2 наиболее продаваемым категориям. Итоговая таблица должна содержать следующие поля: year, quarter_id, product_category_key, english_product_category_name, quarter_sales_amount, quarter_over_quarter_growth_pct.

In [20]:
query = """
with category_sales as (
    select 
        cast(extract(year from c.first_purchase_date) as int) as year,
        ceil(extract(month from c.first_purchase_date) / 3.0) as quarter_id,  
        ps.category_key as product_category_key,
        pc.category_name as english_product_category_name,
        round(sum(s.sales_amount), 1) as quarter_sales_amount
    from 
        adv_works.sales s
    join adv_works.products p 
    	on s.product_key = p.product_key
    join adv_works.product_subcategory ps 
    	on p.subcategory_key = ps.subcategory_key
    join adv_works.product_category pc 
    	on ps.category_key = pc.category_key
    join adv_works.customers c 
    	on s.customer_key = c.customer_key  
    group by 
        year, quarter_id, product_category_key, english_product_category_name
),
top_categories as (
    select 
        product_category_key
    from 
        category_sales
    group by 
        product_category_key
    order by 
        sum(quarter_sales_amount) desc
    limit 2 
)
select 
    cs.year,
    cs.quarter_id,
    cs.product_category_key,
    cs.english_product_category_name,
    cs.quarter_sales_amount,
    round(
        (cs.quarter_sales_amount - coalesce(lag(cs.quarter_sales_amount) 
        over (
        	partition by cs.product_category_key 
        	order by 
        		cs.year, cs.quarter_id), 0)) / 
        nullif(
        coalesce(lag(cs.quarter_sales_amount) 
        over (
        	partition by cs.product_category_key 
        	order by cs.year, cs.quarter_id), 0), 0) * 100, 
        2 ) as quarter_over_quarter_growth_pct
from 
    category_sales cs
join top_categories tc 
	on cs.product_category_key = tc.product_category_key
order by 
    cs.year desc, cs.quarter_id desc, cs.product_category_key;
"""

quarter_df = pd.read_sql(query, conn)

quarter_df

,year,quarter_id,product_category_key,english_product_category_name,quarter_sales_amount,quarter_over_quarter_growth_pct
0,2004,3.0,4,Accessories,18430.2,-83.61
1,2004,2.0,1,Bikes,1945947.7,96.86
2,2004,2.0,4,Accessories,112451.9,-3.27
3,2004,1.0,1,Bikes,988510.9,-26.49
4,2004,1.0,4,Accessories,116257.3,-19.25
5,2003,4.0,1,Bikes,1344659.9,47.10
6,2003,4.0,4,Accessories,143963.0,3.58
7,2003,3.0,1,Bikes,914091.1,-72.08
8,2003,3.0,4,Accessories,138986.3,383.13
9,2003,2.0,1,Bikes,3274193.0,11.96


**Короткий вывод:** Код показывал квартальный рост продаж для топ-2 наиболее продаваемых категорий продуктов, чтобы можно было увидеть динамику продаж для самых продаваемых категорий.

**2. Сравнение будних и выходных (суббота, воскресенье) дней:** Посчитайте продажи в разбивке по годам и дням недели. Определите в какие дни в среднем сумма продаж больше. Определите является ли сумма продаж больше в будние или выходные дни. Итоговая таблица должна содержать следующие поля: year, day_name, is_weekend (где 1 означает выходной а 0 будний день), sales_amount.

In [29]:
query = """
with sales_by_day as (
    select 
        cast(extract(year from c.first_purchase_date) as int) as year,
        to_char(c.first_purchase_date, 'tmday') as day_name,
        case 
            when extract(isodow from c.first_purchase_date) in (6, 7) then 1
            else 0 
        end as is_weekend,
        round(sum(s.sales_amount), 1) as total_sales
    from adv_works.sales s
    join adv_works.customers c 
    	on s.customer_key = c.customer_key
    group by 
        year, day_name, is_weekend
)
select 
    sb.year,
    sb.day_name,
    sb.is_weekend,
    sb.total_sales
from sales_by_day sb
order by 
    sb.total_sales desc;
"""

weekend_df = pd.read_sql(query, conn)

weekend_df

,year,day_name,is_weekend,total_sales
0,2002,wednesday,0,1823393.3
1,2002,tuesday,0,1720327.8
2,2002,thursday,0,1712663.6
3,2002,sunday,1,1711815.7
4,2002,saturday,1,1695324.8
5,2002,monday,0,1672969.1
6,2002,friday,0,1627228.9
7,2003,friday,0,1354545.6
8,2003,thursday,0,1335750.6
9,2003,tuesday,0,1296031.9


**Короткий вывод:** Код выполнил сравнение продаж в будние и выходные дни в разбивке по годам и дням недели. Таким образом, мы получили таблицу с данными, отсортированную по убыванию суммы продаж, что поможет понять, в какие дни недели продажи наиболее высокие и какова разница между будними и выходными днями.